In [21]:
# Import Dependencies
import pandas as pd
import numpy as np
import requests
import json
from appid import api_key
from pprint import pprint
from citipy import citipy
import matplotlib as plt

In [22]:
# Create lists of random longitudes and latitudes
lat = []
lng = []

for x in np.random.randint(-75,75,1200):
    lat.append(x)
    
for y in np.random.randint(-165,165,1200):
    lng.append(y)

# zip together the two lists to obtain pairs of longitude & latitudes in a tuple
lat_lng = tuple(zip(lat,lng))
#print(lat_lng)

In [23]:
# for each latitude-longitude pair we find the nearest city to that pair 
# and put the city and country in the separate lists titled city and country respectively
cities = []
countries =[]
for x, y in lat_lng:
    city = citipy.nearest_city(x, y)
    cityName = city.city_name
    cities.append(cityName)
    countries.append(city.country_code)
    
#print(cities)

In [24]:
# Creat empty lists of temperature, humidity, cloudiness and wind speed
temp = []
hum = []
cloud = []
windsp = []


In [25]:
weather_dict = {
    'City': cities,
    'Latitude': lat,
    'Longitude': lng
}

weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Latitude,Longitude
0,nouna,13,-4
1,pedernales,17,-72
2,hobart,-75,152
3,atuona,-16,-140
4,cape town,-61,-17
...,...,...,...
1195,baoning,32,106
1196,punta arenas,-63,-78
1197,ushuaia,-64,-45
1198,ormara,23,64


In [26]:
weather_data = weather_data.drop_duplicates(subset=["City"], keep="first")

In [27]:
# Create new columns for the other variables
weather_data['Temperature (F)'] = ""
weather_data['Humidity (%)'] = ""
weather_data['Cloudiness (%)'] = ""
weather_data['Wind Speed (mph)'] = ""

weather_data

C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,nouna,13,-4,,,,
1,pedernales,17,-72,,,,
2,hobart,-75,152,,,,
3,atuona,-16,-140,,,,
4,cape town,-61,-17,,,,
...,...,...,...,...,...,...,...
1186,satitoa,-14,-164,,,,
1187,mezen,69,47,,,,
1190,college,66,-153,,,,
1195,baoning,32,106,,,,


In [30]:
# define the base url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"



# Build partial query url
query_url = f"{base_url}appid={api_key}&units={units}&q="

# Build partial query url
#query_url = f"{base_url}&appid={api_key}&units={units}"

# loop through list of cities to obtain the required information for each city
for index, row in weather_data.iterrows():
    
    # get city name from dataframe
    citiName = row['City']
    
    # add keyword to params
    #params['keyword'] = citiName
    
    # API Call
    response = requests.get(query_url + citiName).json()
    
    # Extract results
    #results = response['results']
    
    try:
        weather_data.loc[index, 'Latitude'] = response['coord']['lat']
        weather_data.loc[index, 'Longitude'] = response['coord']['lon']
        weather_data.loc[index, 'Temperature (F)'] = response['main']['temp']
        weather_data.loc[index, 'Humidity (%)'] = response['main']['humidity']
        weather_data.loc[index, 'Cloudiness (%)'] = response['clouds']['all']
        weather_data.loc[index, 'Wind Speed (mph)'] = response['wind']['speed']
        
    except KeyError:
        weather_data.loc[index, 'Latitude'] = ['NaN']
        weather_data.loc[index, 'Longitude'] = ['NaN']
        weather_data.loc[index, 'Temperature (F)'] = ['NaN']
        weather_data.loc[index, 'Humidity (%)'] = ['NaN']
        weather_data.loc[index, 'Cloudiness (%)'] = ['NaN']
        weather_data.loc[index, 'Wind Speed (mph)'] = ['NaN']    
        print("City is missing complete weather data...skip")

C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\U

City is missing complete weather data...skip
City is missing complete weather data...skip


C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-pa

City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is mi

In [31]:
weather_data.head(20)

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,nouna,12.7333,-3.8667,31.1,32,65,3.85
1,pedernales,18.0384,-71.744,25.02,77,70,0.39
2,hobart,-42.8794,147.329,18.27,45,40,2.57
3,atuona,-9.8,-139.033,26.59,73,95,7.45
4,cape town,-33.9258,18.4232,16.1,94,90,4.12
5,hilo,19.7297,-155.09,21.08,69,40,3.09
6,mount gambier,-37.8333,140.767,15,72,90,2.57
7,bolungarvik,NaN,NaN,[NaN],[NaN],[NaN],[NaN]
8,mergui,[NaN],[NaN],[NaN],[NaN],[NaN],[NaN]
9,albany,42.6001,-73.9662,2.53,48,20,3.6


In [37]:
weather_data.count()

#weather_df = weather_data.dropna(how='any')
#weather_df = weather_data[weather_data.duplicated(['City', 'Latitude', 'Longitude'], keep=False)]
#weather_df

City                602
Latitude            602
Longitude           602
Temperature (F)     602
Humidity (%)        602
Cloudiness (%)      602
Wind Speed (mph)    602
dtype: int64

In [ ]:

# loop through list of cities to obtain the required information for each city
for city in cities:
    response = requests.get(query_url + city).json()
print(response)

In [16]:
# Base url for get requests
url = "https://api.openweathermap.org/data/2.5/find?"

query_url = f"{url}&lat={45}&lon={105}&cnt={2}&appid={api_key}"

response = requests.get(query_url)
weather_data = response.json()
pprint(weather_data)

{'cod': '200',
 'count': 2,
 'list': [{'clouds': {'all': 100},
           'coord': {'lat': 45.2432, 'lon': 104.8051},
           'dt': 1619471180,
           'id': 2030890,
           'main': {'feels_like': 266.15,
                    'grnd_level': 854,
                    'humidity': 57,
                    'pressure': 1023,
                    'sea_level': 1023,
                    'temp': 272.36,
                    'temp_max': 272.36,
                    'temp_min': 272.36},
           'name': 'Hashaat',
           'rain': None,
           'snow': None,
           'sys': {'country': 'MN'},
           'weather': [{'description': 'overcast clouds',
                        'icon': '04n',
                        'id': 804,
                        'main': 'Clouds'}],
           'wind': {'deg': 336, 'speed': 7.15}},
          {'clouds': {'all': 100},
           'coord': {'lat': 45.05, 'lon': 105.5667},
           'dt': 1619471181,
           'id': 2028411,
           'main': {'feels_like

In [36]:
df = pd.DataFrame(weather_data['list'])
df

,id,name,coord,main,dt,wind,sys,rain,snow,clouds,weather
0,2030890,Hashaat,"{'lat': 45.2432, 'lon': 104.8051}","{'temp': 272.96, 'feels_like': 265.33, 'temp_m...",1619399157,"{'speed': 11.78, 'deg': 0}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
1,2028411,Ulaanjirem,"{'lat': 45.05, 'lon': 105.5667}","{'temp': 275.89, 'feels_like': 269.67, 'temp_m...",1619399158,"{'speed': 10.27, 'deg': 333}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
2,2031832,Doloon,"{'lat': 44.4245, 'lon': 105.3208}","{'temp': 274.74, 'feels_like': 266.54, 'temp_m...",1619399158,"{'speed': 16.99, 'deg': 271}",{'country': 'MN'},None,None,{'all': 76},"[{'id': 803, 'main': 'Clouds', 'description': ..."
3,2028722,Tsagaan-Ovoo,"{'lat': 45.5072, 'lon': 105.7559}","{'temp': 272.6, 'feels_like': 264.71, 'temp_mi...",1619399157,"{'speed': 12.24, 'deg': 342}",{'country': 'MN'},None,{'1h': 0.312},{'all': 100},"[{'id': 600, 'main': 'Snow', 'description': 'l..."
4,2029321,Sharhulsan,"{'lat': 44.65, 'lon': 104.05}","{'temp': 276.15, 'feels_like': 270.87, 'temp_m...",1619399158,"{'speed': 7.59, 'deg': 322}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
5,2029636,Ongi,"{'lat': 45.45, 'lon': 103.9}","{'temp': 275.51, 'feels_like': 268.85, 'temp_m...",1619399157,"{'speed': 11.41, 'deg': 16}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
6,2029527,Rashaant,"{'lat': 45.3, 'lon': 106.2}","{'temp': 275.37, 'feels_like': 268.5, 'temp_mi...",1619399158,"{'speed': 12.03, 'deg': 329}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
7,2029969,Mongolia,"{'lat': 46, 'lon': 105}","{'temp': 272.9, 'feels_like': 265.28, 'temp_mi...",1619399158,"{'speed': 11.65, 'deg': 354}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
8,2029495,Sangiyn Dalay,"{'lat': 46.0079, 'lon': 104.9421}","{'temp': 273.02, 'feels_like': 265.4, 'temp_mi...",1619399157,"{'speed': 11.81, 'deg': 354}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
9,2031740,Middle Govi,"{'lat': 45.5, 'lon': 106.5}","{'temp': 273.26, 'feels_like': 265.69, 'temp_m...",1619399157,"{'speed': 11.92, 'deg': 330}",{'country': 'MN'},None,None,{'all': 74},"[{'id': 803, 'main': 'Clouds', 'description': ..."
